# Rock Scissor Paper Image Classfication
### train_image = 9109
### test_image = 300

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from PIL import Image
import glob
import os

## ImageDataGenerator

In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image
import numpy as np
import os

# 이미지 증강을 위한 설정
datagen = ImageDataGenerator(
    rotation_range=18,  # 회전 범위 -18도 ~ 18도
    width_shift_range=0.2,  # 가로 이동 범위 20%
    height_shift_range=0.2,  # 세로 이동 범위 20%
    shear_range=0.2,  # 층 밀리기 강도 20%
    zoom_range=0.2,  # 줌 범위 20%
)

image_dir = '/content/drive/MyDrive/DataSet/Aiffel/RSP/train/scissor'

save_dir = '/content/drive/MyDrive/DataSet/Aiffel/RSP/train/scissor2'

for filename in os.listdir(image_dir):
    # 이미지 파일만 처리 (여기서는 .png, .jpg 확장자만 처리)
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_dir, filename)

        # 이미지를 PIL로 열고 NumPy 배열로 변환
        img = np.array(Image.open(image_path))  # PIL 이미지를 NumPy 배열로 변환
        img = img.reshape((1,) + img.shape)  # flow 함수는 4차원 배열을 요구함 (배치 크기 추가)

        # 증강된 이미지를 생성하여 저장
        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir = save_dir, save_prefix='rock', save_format='jpg'):
            i += 1
            if i > 8:  # 원본 이미지에서 증강된 이미지를 3개까지만 생성 (필요에 따라 숫자 조정)
                break


# train Image Unzip

In [4]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/DataSet/Aiffel/RSP/scissor/scissor.zip'
save_path = '/content/drive/MyDrive/DataSet/Aiffel/RSP/scissor'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(save_path)

print(f'파일이 {save_path}에 성공적으로 압축 해제되었습니다.')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DataSet/Aiffel/RSP/scissor/scissor.zip'

# test Image Unzip

In [14]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/DataSet/Aiffel/RSP/test/scissor/scissor.zip'
save_path = '/content/drive/MyDrive/DataSet/Aiffel/RSP/test/scissor'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(save_path)

print(f'파일이 {save_path}에 성공적으로 압축 해제되었습니다.')


파일이 /content/drive/MyDrive/DataSet/Aiffel/RSP/test/scissor에 성공적으로 압축 해제되었습니다.


# 이미지 이름 변경

In [ ]:
def Image_rename(image_dir_path):

    images=glob.glob(image_dir_path + "/*.jpg")

    for name in images:
        directory = os.path.dirname(name)
        base_name = os.path.basename(name)

        filename, zipname = os.path.splitext(base_name)
        new_base_name = filename + "-3" + zipname

        new_name = os.path.join(directory, new_base_name)
        os.rename(name, new_name)

In [ ]:
r_image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/train/new_data/rock"
s_image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/train/new_data/scissor"
p_image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/train/new_data/paper"

Image_rename(r_image_dir_path)
Image_rename(s_image_dir_path)
Image_rename(p_image_dir_path)


# Image Load Resize

In [52]:
from PIL import Image
import glob

def resize_images(img_path):
    images = glob.glob(img_path + "/*.jpg")
    print(len(images), " images to be resized.")

    target_size = (28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.Resampling.LANCZOS)
        new_img.save(img, "JPEG")

In [64]:
image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/train/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

3035  images to be resized.
바위 이미지 resize 완료!


In [65]:
image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/train/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

3048  images to be resized.
가위 이미지 resize 완료!


In [66]:
image_dir_path =  "/content/drive/MyDrive/DataSet/Aiffel/RSP/train/paper"
resize_images(image_dir_path)

print("보자기 이미지 resize 완료!")

3026  images to be resized.
보자기 이미지 resize 완료!


# train, test Dataset

In [60]:
import numpy as np

def load_data(img_path, number_of_data):
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size = 28
    color = 3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0

    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx = idx+1

    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")

    return imgs, labels

In [67]:
image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/train"
(x_train, y_train)=load_data(image_dir_path, 9109)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train_norm.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 9109 입니다.
x_train shape: (9109, 28, 28, 3)
y_train shape: (9109,)


# Modeling

In [76]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dropout

model=keras.models.Sequential()
model.add(keras.layers.Conv2D (32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(Dropout(0.4))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(Dropout(0.4))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_26 (Conv2D)                   │ (None, 26, 26, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 3, 3, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_28 (MaxPooling2D)      │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 113,987 (445.26 KB)

 Trainable params: 113,987 (445.26 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=25)

Epoch 1/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - accuracy: 0.3533 - loss: 1.0885
Epoch 2/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.6504 - loss: 0.7695
Epoch 3/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.8313 - loss: 0.4383
Epoch 4/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.8762 - loss: 0.3136
Epoch 5/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9103 - loss: 0.2391
Epoch 6/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.9204 - loss: 0.2220
Epoch 7/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.9351 - loss: 0.1783
Epoch 8/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.9390 - loss: 0.1653
Epoch 9/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.9483 - loss: 0.1414
Epoch 10/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.9557 - loss: 0.1181
Epoch 11/25
285/285 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.9616 - loss: 0.1067
Epoch 12/25
285/285 ━━━━━━━━━━

# Testing Data Set

In [40]:
image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/test/rock"
resize_images(image_dir_path)

image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/test/scissor"
resize_images(image_dir_path)

image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/test/paper"
resize_images(image_dir_path)

image_dir_path = "/content/drive/MyDrive/DataSet/Aiffel/RSP/test"
(x_test, y_test) = load_data(image_dir_path, 300)

x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

100  images to be resized.
100  images to be resized.
100  images to be resized.
학습데이터(x_train)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


# Model Predict

In [78]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print(f"test_loss: {test_loss}")
print(f"test_accuracy: {test_accuracy}")

10/10 - 0s - 26ms/step - accuracy: 0.4000 - loss: 2.0741
test_loss: 2.0740506649017334
test_accuracy: 0.4000000059604645


## 40%만 나오는 이유
#### 데이터 이미지 중 과도한 노이즈를 발생 시킴
#### Ex) 보자기 : 손날만 보이는 경우